In [1377]:
import csv
import numpy as np
import collections
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import functools
from matplotlib import pylab
import math
%matplotlib inline

num_avg_buckets = 5
VAR = 8

In [1378]:
data = []
counter = 0
with open('train_data.csv', 'r') as f:
    reader = csv.DictReader(f)
    for i,row in enumerate(reader):
        if not row['off'] and not row['def']:
            print(i, row)
            counter += 1
        data.append(row)
print(counter)

0


In [ ]:
grouped_by_game = collections.defaultdict(list)
for line in data:
    grouped_by_game[line['gameid']].append(line)
grouped_by_game = list(grouped_by_game.values())

In [ ]:
for game in grouped_by_game:
    for play_dict in game:
        for key in play_dict:
            try:
                play_dict[key] = int(play_dict[key])
            except:
                pass

In [ ]:
for game in grouped_by_game:
    game.sort(key = lambda x: -int(x['time']))

In [ ]:

total_average_yardage = []
moving_average = []
num_plays_before_calc = 3
for game in grouped_by_game:
    num_total_yards = (0, 0)
    num_plays = (0, 0)
    yard_per_play=[]
    first_team_yards=0
    first_team_plays=0
    second_team_yards=0
    second_team_plays=0

    for i in range(len(game)-1):
        play_dict = game[i]
        alph_first_team = play_dict['off'] if play_dict['off']<play_dict['def'] else play_dict['def']
        next_play_dict = game[i+1]
        if next_play_dict['down'] is not None and play_dict['down'] is not None and play_dict['ydline'] !='' and next_play_dict['ydline'] !='' and next_play_dict['off']==play_dict['off'] and next_play_dict['description'].find('punt')==-1:
            progress = play_dict['ydline']-next_play_dict['ydline']
            if play_dict['off']==alph_first_team:
                first_team_yards+=progress
                first_team_plays+=1
                if first_team_plays>num_plays_before_calc:
                    moving_average.append(first_team_yards/first_team_plays)
            else:
                second_team_yards+=progress
                second_team_plays+=1
                if second_team_plays>num_plays_before_calc:
                    moving_average.append(second_team_yards/second_team_plays)
            yard_per_play.append(progress)

    total_average_yardage.append(first_team_yards/first_team_plays)
    total_average_yardage.append(second_team_yards/second_team_plays)

In [ ]:

total_average_yardage = np.array(total_average_yardage).reshape(len(total_average_yardage), 1)
moving_average = np.array(moving_average).reshape(len(moving_average), 1)

In [ ]:
if 0:
    plt.hist(moving_average, 100)
    plt.show()

In [ ]:

for i,game in enumerate(grouped_by_game):
    first_team_yards=0.0
    first_team_plays=0.0
    second_team_yards=0.0
    second_team_plays=0.0
    for j,play_dict in enumerate(game):
        if j < len(game)-1:
            alph_first_team = play_dict['off'] if play_dict['off']<play_dict['def'] else play_dict['def']
            next_play_dict = game[j+1]
            if next_play_dict['down'] is not None and play_dict['down'] is not None and play_dict['ydline'] !='' and next_play_dict['ydline'] !='' and next_play_dict['off']==play_dict['off'] and next_play_dict['description'].find('punt')==-1:
                progress = play_dict['ydline']-next_play_dict['ydline']
                if play_dict['off']==alph_first_team:
                    first_team_yards+=progress
                    first_team_plays+=1
                    play_dict['avg_yards'] = first_team_yards/first_team_plays
                    play_dict['num_plays_exec'] = first_team_plays
                else:
                    second_team_yards+=progress
                    second_team_plays+=1
                    play_dict['avg_yards'] = second_team_yards/second_team_plays 
                    play_dict['num_plays_exec'] = second_team_plays
            else: #if yards didn't change/it was not a standard play (i.e. field goal)
                if play_dict['off']==alph_first_team and first_team_plays>0:
                    play_dict['avg_yards'] = first_team_yards/first_team_plays
                    play_dict['num_plays_exec'] = first_team_plays
                elif second_team_plays>0:
                    play_dict['avg_yards'] = second_team_yards/second_team_plays
                    play_dict['num_plays_exec'] = second_team_plays

In [ ]:

percentiles = []
for i in range(num_avg_buckets - 1):
    percentiles.append(np.percentile(moving_average, ((i+1) * 100) // num_avg_buckets))
    
def bucket_avg(avg):
    for i in range(num_avg_buckets):
        if avg <= percentiles[i]:
            return i+1
    return num_avg_buckets



if 0:
    twenty_perecentile = np.percentile(moving_average, 20)
    forty_percentile = np.percentile(moving_average, 40)
    sixty_percentile = np.percentile(moving_average, 60)
    eighty_percentile = np.percentile(moving_average, 80)
    fifty_percentile = np.percentile(moving_average, 50)
    _perecentile = np.percentile(moving_average, 20)
    forty_percentile = np.percentile(moving_average, 40)
    sixty_percentile = np.percentile(moving_average, 60)
    eighty_percentile = np.percentile(moving_average, 80)
    fifty_percentile = np.percentile(moving_average, 50)
    percentiles = [0]

In [ ]:
twenty_perecentile,forty_percentile,sixty_percentile,eighty_percentile,fifty_percentile

In [ ]:

def bucket_avg(avg):
    if avg <= twenty_perecentile:
        return 1
    if avg <= forty_percentile:
        return 2
    if avg <= sixty_percentile:
        return 3
    if avg <= eighty_percentile:
        return 4
    return 5
num_avg_bucket = 5
'''
def bucket_avg(avg):
    return 1
    #if avg <= fifty_percentile:
    #    return 1
    #return 2'''

In [ ]:
def put_into_bucket(play_dict):
    yard = play_dict['ydline'] // 10
    down = play_dict['down']
    #if play_dict['togo'] <= 5:
    #    togo = 0
    #if play_dict['togo'] <= 10:
    #    togo = 1
    #else:
    #    togo = 2
    if play_dict['togo'] <= 2:
        togo = 0
    elif play_dict['togo'] <= 4:
        togo = 1
    elif play_dict['togo'] <= 6:
        togo = 2
    elif play_dict['togo'] <= 8:
        togo = 3
    elif play_dict['togo'] <= 10:
        togo = 4
    elif play_dict['togo'] <= 15:
        togo = 5
    elif play_dict['togo'] <= 20:
        togo = 6
    else:
        togo = 7
    '''if play_dict['togo'] <= 4:
        togo = 1
    elif play_dict['togo'] <= 8:
        togo = 3
    elif play_dict['togo'] <= 15:
        togo = 5
    else:
        togo = 7'''
    
    #average yardage
    
    avg_yards = bucket_avg(play_dict['avg_yards'])
    
    if play_dict['off'] < play_dict['def']:
        posession = 1
    else:
        posession = 0
    #return (1, 1, 1, posession)
    return (yard, down, togo, avg_yards, posession)
    #return (1,1,1,1,posession)

In [ ]:
def group_by_score_change(game, original_scores = None):
    if not game:
        return []
    if original_scores is None:
        original_scores = collections.defaultdict(int)
    changed_team = None
    changed_score = None
    for i, play in enumerate(game):
        if original_scores[play['off']] != play['offscore']:
            changed_team = play['off']
            changed_score = play['offscore'] - original_scores[changed_team]
            break
        if original_scores[play['def']] != play['defscore']:
            changed_team = play['def']
            changed_score = play['defscore'] - original_scores[changed_team]
            break     
    
    if changed_team is None:
        return [('', 0,game)]
    
    new_scores = collections.defaultdict(int)
    new_scores[play['off']] = play['offscore']
    new_scores[play['def']] = play['defscore']
    return [(changed_team, changed_score, game[:i+1])] + group_by_score_change(game[i+1:], new_scores)    

In [ ]:
def bucketable(play):
    if not play['down']:
        return False
    if play['down'] < 1:
        return False
    if play['down'] > 4:
        return False
    if not play['togo']:
        return False
    if play['togo'] < 1:
        return False
    if not play['ydline']:
        return False
    if play['ydline'] < 1:
        return False
    if not play['off'] or not play['def']:
        return False
    if play['time'] > 900:
        return False

    if 'num_plays_exec' not in play or play['num_plays_exec'] <= 3:
        return False
    
    return True
    


In [ ]:
def possession(play):
    if play_dict['off'] < play_dict['def']:
        return 1
    else:
        return 0

def bucket_group(group):
    global test
    test = group
    team, diff, group = group
    answer = []
    teams = set()

    for play in group:
        teams.add(play['off'])
        teams.add(play['def'])
    if '' in teams:
        teams.remove('')
    if team:
        #print(teams)
        pos = 1 - sorted(teams).index(team)
    
    if diff != 0:
        group2 = group[:-1]
    else:
        group2 = group
    for i, play in enumerate(group2):
        if play['time'] in (1800, 3600):
            if i == 0 or group2[i-1]['time'] in (1800, 3600):
                answer.append([6])
        if bucketable(play):
            answer.append(put_into_bucket(play))
        else:
            answer.append([6])
            
    
        
    if team:
        '''
        if answer and answer[-1] == [6]:
            answer = answer[:-1]
        if diff == 2:
            answer.append([1, pos])
            answer.append([5, 1-pos])
        elif diff == 3:
            answer.append([0, pos])
            answer.append([5, pos])
        elif diff == 6:
            answer.append([2, pos])
            answer.append([5, pos])
        elif diff == 7:
            answer.append([2, pos])
            answer.append([3, pos])
            answer.append([5, pos])
        elif diff == 8:
            answer.append([2, pos])
            answer.append([4, pos])
            answer.append([5, pos])
        elif diff != 0:
            answer.append([6])
        '''
        play_dict = group[-1]
        if 'avg_yards' in play_dict and play_dict['time'] <= 900:
            avg_yards = bucket_avg(play_dict['avg_yards'])
            '''
            if play_dict['avg_yards'] <= twenty_perecentile:
                avg_yards = 1
            elif play_dict['avg_yards'] <= forty_percentile:
                avg_yards = 2
            elif play_dict['avg_yards'] <= sixty_percentile:
                avg_yards = 3
            elif play_dict['avg_yards'] <= eighty_percentile:
                avg_yards = 4
            else:
                avg_yards = 5'''
            if answer and answer[-1] == [6]:
                answer = answer[:-1]
            if diff == 2:
                answer.append([1, avg_yards, pos])
                answer.append([5, avg_yards, 1-pos])
            elif diff == 3:
                answer.append([0, avg_yards, pos])
                answer.append([5, avg_yards, pos])
            elif diff == 6:
                answer.append([2, avg_yards, pos])
                answer.append([5, avg_yards, pos])
            elif diff == 7:
                answer.append([2, avg_yards, pos])
                answer.append([3, avg_yards, pos])
                answer.append([5, avg_yards, pos])
            elif diff == 8:
                answer.append([2, avg_yards, pos])
                answer.append([4, avg_yards, pos])
                answer.append([5, avg_yards, pos])
            elif diff != 0:
                answer.append([6])
    return answer
        

In [ ]:
def bucket_game(game):
    grouped = group_by_score_change(game)
    answer = []
    for group in grouped:
        answer += bucket_group(group)
    return answer
        


In [ ]:
bucket_sequence = []
for game in grouped_by_game:
    bucket_sequence += bucket_game(game)
    bucket_sequence += [6]

In [ ]:
buckets = bucket_sequence #glue scripts together
len(buckets)

In [ ]:
sum(b != [6] for b in buckets)

In [ ]:
for i in range(len(buckets)):
    if buckets[i] == 6:
        buckets[i] = [6]
    buckets[i] = list(buckets[i])

In [ ]:
counts = collections.defaultdict(collections.Counter)
for i in range(len(buckets) - 1):
    if buckets[i] != [6] and buckets[i+1] != [6]:
        start = buckets[i][:-1]
        end = buckets[i+1][:-2]
        if buckets[i][-1] == buckets[i+1][-1]:
            end.append(0)
        else:
            end.append(1)
    
        counts[tuple(start)][tuple(end)] += 1

In [ ]:
counts2[(3, 2, 3, 2)]

In [ ]:
def smooth_on_feature(counts, feature_num = -1, feature_min = 1, feature_max = num_avg_bucket):
    var = VAR
    counts2 = collections.defaultdict(collections.Counter)
    for bucket in counts:
        mean = bucket[feature_num]
        for feature_val in range(feature_min, feature_max + 1):
            new_bucket = list(bucket)
            new_bucket[feature_num] = feature_val
            new_bucket = tuple(new_bucket)
            if new_bucket in counts:
                weight = 1/math.sqrt(2*math.pi*var) * math.exp(-((feature_val - mean)**2)/ (2 * var))
                for outcome in counts[new_bucket]:
                    counts2[bucket][outcome] += weight * counts[new_bucket][outcome]
    return counts2
    

In [ ]:
counts = smooth_on_feature(counts)

In [ ]:
#remove states we can't support
def completely_in_counts(features, avg, pos_change):
    if pos_change == 1:
        for i in range(1,1 + num_avg_bucket):
            if tuple(list(features) + [i]) not in counts:
                return False
        return True
    return tuple(list(features) +[avg]) in counts

#remove states we can't support
counter = 0
updated = True
while updated:
    updated = False
    for bucket_start in counts:
        avg = bucket_start[-1]
        for bucket_end in counts[bucket_start]:
            end = list(bucket_end)[:-1]
            pos_change = bucket_end[-1]
            if counts[bucket_start][bucket_end] and not completely_in_counts(end, avg, pos_change):
                counts[bucket_start][bucket_end] = 0
                counter += 1
                updated = True
    
print(counter)

In [ ]:
for bucket_start in counts:
    if sum(counts[bucket_start].values()) == 0:
        print(bucket_start)
        raise Exception("Hit Guy up to fix")

In [ ]:
probabilities = collections.defaultdict(dict)
for bucket_start in counts:
    total = sum(counts[bucket_start].values())
    for bucket_end in counts[bucket_start]:
        if counts[bucket_start][bucket_end]:
            probabilities[bucket_start][bucket_end] = counts[bucket_start][bucket_end] / total

In [ ]:
rewards = collections.defaultdict(int)
rewards[(0,0)] = 3
rewards[(1,0)] = 2
rewards[(2,0)] = 6
rewards[(3,0)] = 1
rewards[(4,0)] = 2
for pair in list(rewards.keys()):
    rewards[(pair[0], 1-pair[1])] = -rewards[pair]
for a,b in list(rewards.keys()):
    for i in range(1,num_avg_bucket + 2):
        rewards[(a,i,b)] = rewards[(a,b)]
print(rewards,num_avg_bucket)

In [ ]:
THRESHOLD = 10**-8

@functools.lru_cache(maxsize = None)
def predicted_score_difference(current_state, num_turns_left, off_avg, def_avg):
    if num_turns_left == 0:
        answer = collections.defaultdict(float)
        answer[rewards[current_state]] = 1.0
        return answer
    
    child_score_diffs = collections.defaultdict(float)
    state, pos = current_state[:-1], current_state[-1]
    state = tuple(list(state) + [off_avg])
    #print(state, probabilities[state])
    for child_state in probabilities[state]:
        p = probabilities[state][child_state]
        
        child_state = list(child_state)
        pos_changed = child_state[-1]
        child_state[-1] = ((pos_changed + pos) % 2)

        child_state = tuple(child_state)
        
        if pos_changed:
            child_answer = predicted_score_difference(child_state, num_turns_left - 1, def_avg, off_avg)
        else:
            child_answer = predicted_score_difference(child_state, num_turns_left - 1, off_avg, def_avg)
            
        for score_diff in child_answer:
            child_score_diffs[score_diff] += p * child_answer[score_diff]
    
    answer = collections.defaultdict(float)
    for score_diff in child_score_diffs:
        if child_score_diffs[score_diff] > THRESHOLD:
            answer[score_diff + rewards[current_state]] = child_score_diffs[score_diff]
    return answer

In [ ]:
def compute_metrics_for_game(pdf, start_diff, end_diff):
    actual_diff = end_diff - start_diff
    #print(pdf, actual_diff)
    metric2 = math.log(pdf[actual_diff])
    expected_value = 0
    pairs = []
    for diff in pdf:
        pairs.append([pdf[diff], diff])
        expected_value += diff * pdf[diff]
    metric1 = (expected_value - actual_diff) ** 2
    pairs.sort(reverse = True)
    metric3 = 0
    for pair in pairs[:5]:
        if pair[1] == actual_diff:
            metric3 = 1
    return (metric1, metric2, metric3)

In [ ]:
test_data = []
with open('formatted_test.csv') as f:
    reader = csv.DictReader(f)
    for line in reader:
        test_data.append(line)

In [ ]:
line

In [ ]:
for row in test_data:
    for feature in ('def_team_avg_yards', 'off_team_avg_yards'):
        row[feature] = float(row[feature])
    for feature in ('down', 'final_diff', 'third_diff', 'togo', 'yard_line'):
        row[feature] = int(row[feature])
    

In [ ]:
def put_into_bucket_test(play_dict):
    play_dict['ydline'] = play_dict['yard_line']
    play_dict['off'] = 'a'
    play_dict['def'] = 'a'
    play_dict['avg_yards'] = 5
    if play_dict['yard_line'] == -1 or play_dict['down'] == -1 or play_dict['togo'] == -1:
        return (5,)
    bucket = put_into_bucket(play_dict)
    return bucket[:-2]
    #return (yard, down, togo, avg_yards, posession)
    


In [ ]:
metrics = [0,0,0]
wrong = []
for game in test_data:
    start_bucket = tuple(list(put_into_bucket_test(game)) + [0])
    
    pdf = predicted_score_difference(start_bucket, 43, bucket_avg(game['off_team_avg_yards']),bucket_avg(game['def_team_avg_yards']))
    #print(sum(pdf.values()))
    for i, metric in enumerate(compute_metrics_for_game(pdf, game['third_diff'], game['final_diff'])):
        metrics[i] += metric
        if i == 2 and metric == 0:
            wrong.append(game)

In [ ]:
for i in range(3):
    print(metrics[i] / len(test_data))

In [ ]:
game

In [ ]:
test_data.index(game)

In [ ]:
len(test_data)

In [ ]:
pdf = predicted_score_difference((2, 1,1, 0), 43, 2, 1)
score_diff = []
probability = []
smallest = sorted(pdf.items())[0][0]
largest = sorted(pdf.items())[-1][0]
print(smallest, largest)
for s in range(-40, 41):
    p = pdf[s]
    score_diff.append(s)
    probability.append(p)
pylab.rcParams['figure.figsize'] = (25.0, 16.0)
indexes = np.arange(len(score_diff))
width = 1

plt.bar(indexes, probability, width)
plt.xticks(indexes + width * 0.5, score_diff)
plt.show()

In [ ]:
probabilities[((1, 1,1, 1))]

In [ ]:
num_avg_bucket